<a href="https://colab.research.google.com/github/khanhlinhbk/Semantic-Text-Similarity-With-Bert/blob/master/Bert%2BCNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers

     |████████████████████████████████| 2.5MB 13.7MB/s 
     |████████████████████████████████| 3.3MB 53.9MB/s 
     |████████████████████████████████| 901kB 26.3MB/s 


In [ ]:
!pip install ipython-autotime

%load_ext autotime

time: 124 µs (started: 2021-06-20 09:11:47 +00:00)


In [ ]:

import numpy as np
import pandas as pd
import tensorflow as tf
import transformers

time: 3.51 s (started: 2021-06-20 09:11:50 +00:00)


In [ ]:
print(transformers.__version__)

4.7.0
time: 832 µs (started: 2021-06-20 09:11:56 +00:00)


In [ ]:
import pickle
import tempfile
from tensorflow.keras.models import Sequential, load_model, save_model, Model
from tensorflow.keras.layers import Dense

# Hotfix function
def make_keras_picklable():
    def __getstate__(self):
        model_str = ""
        with tempfile.NamedTemporaryFile(suffix='.hdf5', delete=True) as fd:
            save_model(self, fd.name, overwrite=True)
            model_str = fd.read()
        d = {'model_str': model_str}
        return d

    def __setstate__(self, state):
        with tempfile.NamedTemporaryFile(suffix='.hdf5', delete=True) as fd:
            fd.write(state['model_str'])
            fd.flush()
            model = load_model(fd.name)
        self.__dict__ = model.__dict__


    cls = Model
    cls.__getstate__ = __getstate__
    cls.__setstate__ = __setstate__

# Run the function
make_keras_picklable()

time: 11.5 ms (started: 2021-06-20 09:12:09 +00:00)


In [ ]:
from google.colab import files
uploaded = files.upload()

Saving questions.csv to questions.csv
time: 14min 2s (started: 2021-06-20 09:12:12 +00:00)


In [ ]:
import io
df = pd.read_csv(io.StringIO(uploaded['questions.csv'].decode('utf-8')))
df

,id,qid1,qid2,question1,question2,is_duplicate
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0
...,...,...,...,...,...,...
404346,404346,789792,789793,How many keywords are there in the Racket prog...,How many keywords are there in PERL Programmin...,0
404347,404347,789794,789795,Do you believe there is life after death?,Is it true that there is life after death?,1
404348,404348,789796,789797,What is one coin?,What's this coin?,0
404349,404349,789798,789799,What is the approx annual cost of living while...,I am having little hairfall problem but I want...,0


time: 1.1 s (started: 2021-06-20 09:27:02 +00:00)


In [ ]:
print(f"Total train samples : {df.shape[0]}")

Total train samples : 404351
time: 840 µs (started: 2021-06-20 09:27:06 +00:00)


In [ ]:
df.drop(['id', 'qid1', 'qid2'], axis=1, inplace=True)
df.columns = ['question1', 'question2', 'label']

time: 27.8 ms (started: 2021-06-20 09:27:13 +00:00)


In [ ]:
train, validate, test = \
              np.split(df.sample(frac=1, random_state=42), 
                       [int(.6*len(df)), int(.8*len(df))])

time: 176 ms (started: 2021-06-20 09:27:15 +00:00)


In [ ]:

train

,question1,question2,label
120567,How does the Boggart work?,What would the boggart of a boggart be?,0
324466,What is difference between project manager and...,What are the differences between project manag...,0
398558,What hotel in Jabalpur would be safe for unmar...,What hotel in Allahabad would be safe for unma...,0
339914,What is stronger - Super Saiyan 4 or Super Sai...,How does Gohan turn into Super Saiyan 2?,0
185732,How do I fill in Address Line 1 and Address Li...,How do I register desired web address?,0
...,...,...,...
92851,What elements of OO design are most prone to a...,What is the easiest way to delete an array ele...,0
371102,What weapons are used in spiritual warfare?,What is spiritual warfare? What are the weapon...,1
157072,How can I reduce fat from right side of face ?,How can I loose fat from my face?,1
123337,How can I have 200 calories per day?,"If I eat once a day and during that one time, ...",0


time: 37.7 ms (started: 2021-06-20 09:27:19 +00:00)


In [ ]:
test

,question1,question2,label
301066,How should I talk to the girl I like?,How do you go up and talk to a girl you like?,1
51450,Who has the best military?,Who has the best military in the world?,1
325180,How does one become a mercenary?,How do I become a mercenary?,1
191549,How did buying and/or selling a home become so...,How do I market or increase visibility of a bu...,0
281839,Why use demand draft?,What is the difference between chalan and dema...,0
...,...,...,...
259178,If universe is expanding without a limit and d...,How is dark/vacuum energy created with the uni...,1
365838,What is the superstition behind a twitching ri...,What is the superstition of the lower left eye...,1
131932,Does Quora form intellectual bubbles?,Who owns the intellectual property of Quora?,0
146867,I just had a very bad break up. My boyfriend l...,What did you do when your ex moved on?,0


time: 21 ms (started: 2021-06-20 09:27:37 +00:00)


In [ ]:
print(train.shape)

(242610, 3)
time: 912 µs (started: 2021-06-20 09:27:23 +00:00)


In [ ]:
print(validate.shape)

(80870, 3)
time: 929 µs (started: 2021-06-20 09:27:43 +00:00)


In [ ]:
print(test.shape)

(80871, 3)
time: 829 µs (started: 2021-06-20 09:27:46 +00:00)


In [ ]:

validate

,question1,question2,label
171036,Do gravitational waves affect the flow of time?,Do gravitational waves affect time?,1
3724,What are some symptoms of eccentric and concen...,How do concentric and eccentric contraction co...,1
189804,What kind of questions are asked in technical ...,How do I change my profile in joinindianarmy.n...,0
19942,How can I lose 20 lbs super fast to audition f...,How do you lose 20 lbs fast?,0
120562,What is the most embarrassing thing you though...,What are the most embarrassing things that kid...,0
...,...,...,...
205349,Which companies sponsor H1B visa in Pasadena?,Which companies sponsor H1B visas?,0
142526,What is the port number for whatsapp?,What is the port number of gmail?,0
311872,Is it bad for me or my dog to give him too muc...,I love my dog too much and thus give her kisse...,0
59941,Could the Donald Trump presidential run be a j...,Can't this election be a conspiracy where Trum...,1


time: 21.2 ms (started: 2021-06-20 09:27:48 +00:00)


In [ ]:

test

,question1,question2,label
301066,How should I talk to the girl I like?,How do you go up and talk to a girl you like?,1
51450,Who has the best military?,Who has the best military in the world?,1
325180,How does one become a mercenary?,How do I become a mercenary?,1
191549,How did buying and/or selling a home become so...,How do I market or increase visibility of a bu...,0
281839,Why use demand draft?,What is the difference between chalan and dema...,0
...,...,...,...
259178,If universe is expanding without a limit and d...,How is dark/vacuum energy created with the uni...,1
365838,What is the superstition behind a twitching ri...,What is the superstition of the lower left eye...,1
131932,Does Quora form intellectual bubbles?,Who owns the intellectual property of Quora?,0
146867,I just had a very bad break up. My boyfriend l...,What did you do when your ex moved on?,0


time: 21.5 ms (started: 2021-06-20 09:27:55 +00:00)


In [ ]:
print("Train Target Distribution")
print(train.label.value_counts())

Train Target Distribution
0    153059
1     89551
Name: label, dtype: int64
time: 6.75 ms (started: 2021-06-20 09:27:58 +00:00)


In [ ]:
print("Validate Target Distribution")
print(validate.label.value_counts())

Validate Target Distribution
0    51006
1    29864
Name: label, dtype: int64
time: 3.93 ms (started: 2021-06-20 09:28:01 +00:00)


In [ ]:
print("Test Target Distribution")
print(test.label.value_counts())

Test Target Distribution
0    50980
1    29891
Name: label, dtype: int64
time: 4.66 ms (started: 2021-06-20 09:28:03 +00:00)


In [ ]:
y_train = tf.keras.utils.to_categorical(train.label, num_classes=2)
y_validate = tf.keras.utils.to_categorical(validate.label, num_classes=2)
y_test = tf.keras.utils.to_categorical(test.label, num_classes=2)

time: 7.72 ms (started: 2021-06-20 09:28:07 +00:00)


In [ ]:
max_length = 128  # Maximum length of input sentence to the model.
batch_size = 32
epochs = 5

time: 1.04 ms (started: 2021-06-20 09:28:09 +00:00)


In [ ]:
class BertSemanticDataGenerator(tf.keras.utils.Sequence):
    """Generates batches of data.

    Args:
        sentence_pairs: Array of premise and hypothesis input sentences.
        labels: Array of labels.
        batch_size: Integer batch size.
        shuffle: boolean, whether to shuffle the data.
        include_targets: boolean, whether to incude the labels.

    Returns:
        Tuples `([input_ids, attention_mask, `token_type_ids], labels)`
        (or just `[input_ids, attention_mask, `token_type_ids]`
         if `include_targets=False`)
    """

    def __init__(
        self,
        sentence_pairs,
        labels,
        batch_size=batch_size,
        shuffle=True,
        include_targets=True,
    ):
        self.sentence_pairs = sentence_pairs
        self.labels = labels
        self.shuffle = shuffle
        self.batch_size = batch_size
        self.include_targets = include_targets
        # Load our BERT Tokenizer to encode the text.
        # We will use base-base-uncased pretrained model.
        self.tokenizer = transformers.BertTokenizer.from_pretrained(
            "bert-base-uncased", do_lower_case=True
        )
        self.indexes = np.arange(len(self.sentence_pairs))
        self.on_epoch_end()

    def __len__(self):
        # Denotes the number of batches per epoch.
        return len(self.sentence_pairs) // self.batch_size

    def __getitem__(self, idx):
        # Retrieves the batch of index.
        indexes = self.indexes[idx * self.batch_size : (idx + 1) * self.batch_size]
        sentence_pairs = self.sentence_pairs[indexes]

        # With BERT tokenizer's batch_encode_plus batch of both the sentences are
        # encoded together and separated by [SEP] token.
        encoded = self.tokenizer.batch_encode_plus(
            sentence_pairs.tolist(),
            add_special_tokens=True,
            max_length=max_length,
            return_attention_mask=True,
            return_token_type_ids=True,
            pad_to_max_length=True,
            return_tensors="tf",
        )

        # Convert batch of encoded features to numpy array.
        input_ids = np.array(encoded["input_ids"], dtype="int32")
        attention_masks = np.array(encoded["attention_mask"], dtype="int32")
        token_type_ids = np.array(encoded["token_type_ids"], dtype="int32")

        # Set to true if data generator is used for training/validation.
        if self.include_targets:
            labels = np.array(self.labels[indexes], dtype="int32")
            return [input_ids, attention_masks, token_type_ids], labels
        else:
            return [input_ids, attention_masks, token_type_ids]

    def on_epoch_end(self):
        # Shuffle indexes after each epoch if shuffle is set to True.
        if self.shuffle:
            np.random.RandomState(42).shuffle(self.indexes)

time: 25.9 ms (started: 2021-06-20 09:28:12 +00:00)


In [ ]:
from tensorflow import keras
from keras.layers import Dense
from transformers import BertConfig;

time: 223 ms (started: 2021-06-20 09:28:16 +00:00)


In [ ]:
# Create the model under a distribution strategy scope.
strategy = tf.distribute.MirroredStrategy()

with strategy.scope():
    # Encoded token ids from BERT tokenizer.
    input_ids = tf.keras.layers.Input(
        shape=(max_length,), dtype=tf.int32, name="input_ids"
    )
    # Attention masks indicates to the model which tokens should be attended to.
    attention_masks = tf.keras.layers.Input(
        shape=(max_length,), dtype=tf.int32, name="attention_masks"
    )
    # Token type ids are binary masks identifying different sequences in the model.
    token_type_ids = tf.keras.layers.Input(
        shape=(max_length,), dtype=tf.int32, name="token_type_ids"
    )
    config = BertConfig.from_pretrained('bert-base-uncased', output_hidden_states=True, output_attentions=True)
    # Loading pretrained BERT model.
    bert_model = transformers.TFBertModel.from_pretrained("bert-base-uncased",config=config)
    # Freeze the BERT model to reuse the pretrained features without modifying them.
    bert_model.trainable = True

    bert_output = bert_model(
    input_ids, attention_mask=attention_masks, token_type_ids=token_type_ids
    )
    bert_output = tf.concat([bert_output[2][-i] for i in range(4)], axis=-1)
   
  
   
    out = tf.keras.layers.Dropout(0.15)(bert_output)
    
    out = tf.keras.layers.Conv1D(768, 2,padding='same', activation="relu")(out)
    out = tf.keras.layers.LeakyReLU()(out)
    out= tf.keras.layers.MaxPooling1D(pool_size=2)(out)
    out = tf.keras.layers.Conv1D(64, 2,padding='same', activation="relu")(out)
    out = tf.keras.layers.Dense(16, activation='relu')(out)
    out = tf.keras.layers.Flatten()(out)
    out = tf.keras.layers.Dense(2, activation='softmax')(out)
    model = tf.keras.models.Model(
        inputs=[input_ids, attention_masks, token_type_ids], outputs=out
    )

    model.compile(
        optimizer=tf.keras.optimizers.Adam(1e-5),
        loss="binary_crossentropy",
        metrics=["acc"],
    )


print(f"Strategy: {strategy}")
model.summary()

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)


Some layers from the model checkpoint at bert-base-uncased were not used when initializing TFBertModel: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: while/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: while/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experi

In [ ]:
train_data = BertSemanticDataGenerator(
    train[["question1", "question2"]].values.astype("str"),
    y_train,
    batch_size=batch_size,
    shuffle=True,
)
valid_data = BertSemanticDataGenerator(
    validate[["question1", "question2"]].values.astype("str"),
    y_validate,
    batch_size=batch_size,
    shuffle=False,
)


time: 7.82 s (started: 2021-06-20 09:29:31 +00:00)


In [ ]:
history = model.fit(
    train_data,
    validation_data=valid_data,
    epochs=epochs,
    use_multiprocessing=True,
    workers=-1,
)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2111: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


Epoch 1/5
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
7581/7581 [==============================] - ETA: 0s - loss: 0.3288 - acc: 0.8493

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


7581/7581 [==============================] - 4150s 544ms/step - loss: 0.3288 - acc: 0.8493 - val_loss: 0.2656 - val_acc: 0.8824
Epoch 2/5
7581/7581 [==============================] - 4116s 543ms/step - loss: 0.2233 - acc: 0.9065 - val_loss: 0.2398 - val_acc: 0.8961
Epoch 3/5
7581/7581 [==============================] - 4122s 544ms/step - loss: 0.1595 - acc: 0.9361 - val_loss: 0.2519 - val_acc: 0.8979
Epoch 4/5
7581/7581 [==============================] - 4119s 543ms/step - loss: 0.1105 - acc: 0.9580 - val_loss: 0.2912 - val_acc: 0.8988
Epoch 5/5
2790/7581 [==========>...................] - ETA: 38:55 - loss: 0.0731 - acc: 0.9733

In [ ]:
test_data = BertSemanticDataGenerator(
    test[["question1", "question2"]].values.astype("str"),
    y_test,
    batch_size=batch_size,
    shuffle=False,
)
model.evaluate(test_data, verbose=1)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2111: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


2527/2527 [==============================] - 452s 179ms/step - loss: 0.3077 - acc: 0.8989


[0.30773791670799255, 0.8988672494888306]

time: 7min 36s (started: 2021-06-19 22:37:49 +00:00)


In [ ]:
labels = [1, 0]

time: 1.16 ms (started: 2021-06-19 22:45:26 +00:00)


In [ ]:
def check_similarity(sentence1, sentence2):
    sentence_pairs = np.array([[str(sentence1), str(sentence2)]])
    test_data = BertSemanticDataGenerator(
        sentence_pairs, labels=None, batch_size=1, shuffle=False, include_targets=False,
    )

    proba = model.predict(test_data)[0]
    idx = np.argmax(proba)
    proba = f"{proba[idx]: .2f}%"
    pred = labels[idx]
    if pred == 1:
      label = "Similar"
    else:
      label = "Not Similar"
    print("Prediction: {0}\nProbability: {1}\n\n".format(label, proba))

time: 7.72 ms (started: 2021-06-19 22:45:26 +00:00)


In [ ]:
sentence1 = "Two women are observing something together."
sentence2 = "Two women are standing with their eyes closed."
check_similarity(sentence1, sentence2)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2111: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


Prediction: Similar
Probability:  0.82%


time: 8.33 s (started: 2021-06-19 22:45:26 +00:00)


In [ ]:
sentence1 = "A soccer game with multiple males playing"
sentence2 = "Some men are playing a football"
check_similarity(sentence1, sentence2)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2111: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


Prediction: Similar
Probability:  0.96%


time: 3.83 s (started: 2021-06-19 22:45:35 +00:00)


In [ ]:
sentence1 = "How old are you?"
sentence2 = "What is your age?"
check_similarity(sentence1, sentence2)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


Prediction: Similar
Probability:  0.99%


time: 3.83 s (started: 2021-06-19 22:45:38 +00:00)


/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2111: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [ ]:
sentence1 = "Does that dog belong to you?"
sentence2 = "Is that dog yours?"
check_similarity(sentence1, sentence2)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


Prediction: Similar
Probability:  0.97%


time: 4.07 s (started: 2021-06-19 22:45:42 +00:00)


/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2111: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [ ]:
sentence1 = "Astrology: I am a Capricorn Sun Cap moon and cap rising...what does that say about me?"
sentence2 = "I'm a triple Capricorn (Sun, Moon and ascendant in Capricorn) What does this say about me?"
check_similarity(sentence1, sentence2)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


Prediction: Not Similar
Probability:  0.90%


time: 4.09 s (started: 2021-06-19 22:45:46 +00:00)


/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2111: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [ ]:
sentence1 = "you are a dog"
sentence2 = "you look like a dog"
check_similarity(sentence1, sentence2)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


Prediction: Similar
Probability:  1.00%


time: 4.08 s (started: 2021-06-19 22:45:50 +00:00)


/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2111: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,
